In [ ]:
# %pip install --upgrade gradio
# %pip install pydantic
# %pip install langchain

# generate a chatbot front end using Gradio

In ChatGPT we are accustomed to having a user interface that displays our chat history and the responses from the AI.

We will recreate that functionality using Gradio

https://www.gradio.app/guides/creating-a-chatbot-fast
https://www.gradio.app/docs/gradio/chatbot

## generate Auth
API requests in Domo must be authenticated

In [ ]:
import importlib
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj

DOMO_INSTANCE = 'domo-community'

auth = await dxdj.generate_domo_auth(
    domo_instance = DOMO_INSTANCE,
    domojupyter_fn = dj) 

auth.user_id

## Format chat history correctly
Gradio expects history : `List[str]` and the incoming message as `message: str`

In [ ]:
import gradio as gr

async def generate_state():
    res = await auth.who_am_i()
    
    
    return {'file_path' : 'history.json',
             'user_id' : res.response['id'],
             'display_name' : res.response['displayName']
            }

state = await generate_state()
state

In [ ]:
import json

def save_history(history, ):
    print(f'saving history to {file_path}')

    with open(file_path, 'w') as f:
        json.dump( history, f)


### instantiate endpoint

In [ ]:
import models.model as model
importlib.reload(model)

endpoint = model.EndpointHandler._from_creds_account(domo_instance = DOMO_INSTANCE)


## reset chat history

In [ ]:
from gradio import ChatMessage

system_message = "your name is beth, you are a software engineer, but try to be helpful when you can"

history = [
    ChatMessage( "user", "What is the capital of France?"),
    ChatMessage( "assistant" , "Paris")
    
]

message = "And what is its longest street?"

In [ ]:
from typing import List

def prep_new_history(history):
    nh = []
    nh.append(ChatMessage(role = "sytem", content = system_message))
    nh += history  
    return nh
    
#     # return [a.__dict__ for a in nh]
    
def chat(message, 
         history,
         # state = None,
         debug_api : bool = True):
        
    nh = prep_new_history(history = history)

    
    res = endpoint.chat(history = nh,
                        message = message,
                        debug_api = debug_api)
    
    history.append(ChatMessage(role = 'user', content = message))

    output = res.response['output'] 
    
    history.append(ChatMessage(role="assistant", content=output))
    
    return "", history #, state
               
chat(message = message,
     history = history, 
     debug_api = False
)

In [ ]:
import gradio as gr
from functools import partial

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    
    state = gr.State()

    msg.submit(
        chat,
        [msg, chatbot,],
        [msg, chatbot]
    )
    
demo.launch(share = True)